In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [11]:
driver = webdriver.Chrome()
driver.get('https://www.linkedin.com/login')

In [13]:
driver.find_element('id','username').send_keys('matheusufmg23@gmail.com')
driver.find_element('id','password').send_keys('ufmg2024')
driver.find_element(By.XPATH, "//button").click()

In [14]:
def sanitizeString(value):
    value=str(value)
    value=value.replace('<!-- -->', '')
    value=value.replace('<span class="visually-hidden">', '')
    value=value.replace('</span>', '')
    return value.split(" · ")

In [15]:
def extractProfileData(url):
    driver.get(url)
    htmlPage = driver.page_source
    site = BeautifulSoup(htmlPage)
    
    main=site.find("main")
    
    
    element_present = EC.presence_of_element_located((By.TAG_NAME, 'ul'))
    WebDriverWait(driver, 10).until(element_present)
    
    ul=main.find("ul")
    [x.extract() for x in ul.findAll('ul')]
    element_present = EC.presence_of_element_located((By.TAG_NAME, 'li'))
    WebDriverWait(driver, 10).until(element_present)
    li = ul.findChildren('li')
    experiencias= []
    for x in li:
        span = x.findChildren("span", "visually-hidden")
        
        if(sanitizeString(span[len(span)-1])[0].find("LinkedIn helped me get this job")>=0):
            span.pop()
            
        if len(span) < 4:
            value1=(sanitizeString(span[0]))
            value2=(sanitizeString(span[1]))
            experiencias.append([value1[0],value2[0], value2[1], ''])
        else:
            #print(span[0].renderContents()) ## informação não vai ser usada
            value1=(sanitizeString(span[1]))  ## [nome da empresa , tipo de contrato]
            value2=(sanitizeString(span[2]))  ## tempo de trabalho
            
            value3=(sanitizeString(span[3])) if len(span)>=4 else ['','']  ## Cidade, País, remoto ou não
            
            tempo = "" if value2[0].find('Present')>=0 else value2[1]
        
            if(len(value1)>1):
                if(value1[1]!="Freelance"):
                    experiencias.append([value1[0],value1[1], tempo, value3[0]])
            else:
                experiencias.append([value1[0],'', tempo, value3[0]])
    return experiencias

In [29]:
df = pq.read_table(source='./profiles-url-itau.parquet').to_pandas()
print(len(df['url']))
count = 0
dataExtracted=[]
until = len(df['url'])
while count < until:
    try:
        profile=df['url'][count]
        endUrl=(profile.index('?mini'))
        profile=profile[:endUrl]+"/details/experience/"
        result = extractProfileData(profile)
        dataExtracted.append(result)
        print(count+1 ,' de ',until)
    except:
      print(count+1 ,' de ',until,"An exception occurred")
    count+=1



690
1  de  690
2  de  690
3  de  690
4  de  690
5  de  690
6  de  690 An exception occurred
7  de  690
8  de  690 An exception occurred
9  de  690
10  de  690
11  de  690
12  de  690
13  de  690
14  de  690 An exception occurred
15  de  690
16  de  690 An exception occurred
17  de  690 An exception occurred
18  de  690
19  de  690
20  de  690 An exception occurred
21  de  690
22  de  690 An exception occurred
23  de  690 An exception occurred
24  de  690 An exception occurred
25  de  690
26  de  690 An exception occurred
27  de  690 An exception occurred
28  de  690 An exception occurred
29  de  690
30  de  690
31  de  690 An exception occurred
32  de  690
33  de  690 An exception occurred
34  de  690 An exception occurred
35  de  690 An exception occurred
36  de  690
37  de  690
38  de  690 An exception occurred
39  de  690 An exception occurred
40  de  690 An exception occurred
41  de  690 An exception occurred
42  de  690 An exception occurred
43  de  690
44  de  690
45  de  690
46 

In [30]:
print(len(dataExtracted))

271


In [31]:
profiles_schema = pa.schema([
('data', pa.large_list(pa.int8())),
])
paProfiles = pa.array(dataExtracted)
batch = pa.RecordBatch.from_arrays(
    [paProfiles],
    names=profiles_schema.names
)

print(len(batch))
print(batch[0][0])
print(batch[0][1])

271
[['Liberty Seguros Brasil', 'Full-time', '', 'São Paulo, Brasil'], ['Itaú Unibanco', 'Full-time', '3 yrs 4 mos', 'São Paulo, Brasil']]
[['Stone', 'Full-time', '', 'Belo Horizonte, Minas Gerais, Brazil'], ['Itaú Unibanco', 'Full-time', '11 mos', 'Remote'], ['Hotmart', 'Full-time', '3 yrs 6 mos', 'Belo Horizonte, Minas Gerais, Brazil'], ['Sigga Workforce Technologies', 'Full-time', '9 mos', 'Belo Horizonte Area, Brazil'], ['Toshiba America do Sul Ltda', '', '5 yrs 3 mos', 'Contagem - Minas Gerais - Brazil'], ['Geeks Group', '', '10 mos', 'Belo Horizonte - Minas Gerais'], ['TEM Industria e Comercio Ltda', '', '8 yrs 2 mos', 'Belo Horizonte e Região, Brasil']]


In [32]:

table = pa.Table.from_batches([batch])
print(len(paProfiles))
pq.write_table(table, './profiles-data-itau.parquet')

271


In [ ]:
experiencias= []
for x in li:
    span = x.findChildren("span", "visually-hidden")
    #print(span[0].renderContents()) ## informação não vai ser usada
    value1=(sanitizeString(span[1]))  ## [nome da empresa , tipo de contrato]
    value2=(sanitizeString(span[2]))  ## tempo de trabalho
    value3=(sanitizeString(span[3]))  ## Cidade, País, remoto ou não

    tempo = "" if value2[0].find('Present')>=0 else value2[1]

    
    if(value1[1]!="Freelance"):
        experiencias.append([value1[0],value1[1], tempo, value3[0]])


In [138]:
print(experiencias)
number_of_experiences = len(experiencias)
print(number_of_experiences)

[['Incentive.me', 'Full-time', '', 'Rio de Janeiro, Brazil'], ['Arena', 'Full-time', '1 yr 5 mos', 'San Francisco, California, United States'], ['Vivo (Telefônica Brasil)', 'Full-time', '1 yr 2 mos', 'Curitiba, Paraná, Brazil'], ['CNA', 'Part-time', '1 yr', 'Rio de Janeiro, Brazil'], ['Universidade Federal Fluminense', 'Apprenticeship', '1 yr 6 mos', 'Rio de Janeiro Area, Brazil'], ['Instituto Federal de Educação, Ciência e Tecnologia Fluminense', 'Internship', '1 yr 6 mos', 'Rio de Janeiro Area, Brazil']]
6


In [26]:
profiles_schema = pa.schema([
('number_of_experiences', pa.string()),
])

https://www.linkedin.com/in/alanpfabricio


In [107]:
teste='apbe<span class="visually-hidden">'
print((teste).replace('<span class="visually-hidden">', ''))


apbe
